Machine Vision Lab 2

In [ ]:
from PIL import Image

filename = "Correction_Tape_Original.png"

with Image.open(filename) as img:
    img.load()

# img.format ##  format = PNG
# img.size ## size = (1280, 960)
# img.mode ## mode = RGB